# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 1 2022 12:47PM,249799,12,HH-37896,Hush Hush,Released
1,Nov 1 2022 12:47PM,249799,12,HH-37897,Hush Hush,Released
2,Nov 1 2022 12:47PM,249799,12,HH-37898,Hush Hush,Released
3,Nov 1 2022 12:47PM,249799,12,HH-37899,Hush Hush,Released
4,Nov 1 2022 12:47PM,249799,12,HH-37900,Hush Hush,Released
5,Nov 1 2022 12:47PM,249799,12,HH-37901,Hush Hush,Released
6,Nov 1 2022 12:47PM,249799,12,HH-37902,Hush Hush,Released
7,Nov 1 2022 12:34PM,249798,12,HH8265,Hush Hush,Released
8,Nov 1 2022 12:20PM,249797,19,ADV80007215,"AdvaGen Pharma, Ltd",Released
9,Nov 1 2022 12:20PM,249797,19,ADV80007217,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,249795,Released,1
18,249797,Executing,2
19,249797,Released,9
20,249798,Released,1
21,249799,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249794,NaN,NaN,1.0
249795,NaN,NaN,1.0
249797,NaN,2.0,9.0
249798,NaN,NaN,1.0
249799,NaN,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249685,58.0,0.0,7.0
249716,2.0,1.0,0.0
249750,31.0,24.0,4.0
249753,0.0,1.0,0.0
249757,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249685,58,0,7
249716,2,1,0
249750,31,24,4
249753,0,1,0
249757,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249685,58,0,7
1,249716,2,1,0
2,249750,31,24,4
3,249753,0,1,0
4,249757,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249685,58,,7
1,249716,2,1,
2,249750,31,24,4
3,249753,,1,
4,249757,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 1 2022 12:47PM,249799,12,Hush Hush
7,Nov 1 2022 12:34PM,249798,12,Hush Hush
8,Nov 1 2022 12:20PM,249797,19,"AdvaGen Pharma, Ltd"
19,Nov 1 2022 12:05PM,249795,21,"NBTY Global, Inc."
20,Nov 1 2022 12:05PM,249794,21,"NBTY Global, Inc."
21,Nov 1 2022 12:04PM,249793,21,"NBTY Global, Inc."
22,Nov 1 2022 11:30AM,249788,12,"SD Head USA, LLC"
23,Nov 1 2022 11:21AM,249786,10,Methapharm-G
26,Nov 1 2022 11:21AM,249786,10,"Methapharm, Inc."
28,Nov 1 2022 11:13AM,249783,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 1 2022 12:47PM,249799,12,Hush Hush,,,7
1,Nov 1 2022 12:34PM,249798,12,Hush Hush,,,1
2,Nov 1 2022 12:20PM,249797,19,"AdvaGen Pharma, Ltd",,2,9
3,Nov 1 2022 12:05PM,249795,21,"NBTY Global, Inc.",,,1
4,Nov 1 2022 12:05PM,249794,21,"NBTY Global, Inc.",,,1
5,Nov 1 2022 12:04PM,249793,21,"NBTY Global, Inc.",,,1
6,Nov 1 2022 11:30AM,249788,12,"SD Head USA, LLC",,,1
7,Nov 1 2022 11:21AM,249786,10,Methapharm-G,,4,1
8,Nov 1 2022 11:21AM,249786,10,"Methapharm, Inc.",,4,1
9,Nov 1 2022 11:13AM,249783,10,"Citieffe, Inc.",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 1 2022 12:47PM,249799,12,Hush Hush,7,,
1,Nov 1 2022 12:34PM,249798,12,Hush Hush,1,,
2,Nov 1 2022 12:20PM,249797,19,"AdvaGen Pharma, Ltd",9,2,
3,Nov 1 2022 12:05PM,249795,21,"NBTY Global, Inc.",1,,
4,Nov 1 2022 12:05PM,249794,21,"NBTY Global, Inc.",1,,
5,Nov 1 2022 12:04PM,249793,21,"NBTY Global, Inc.",1,,
6,Nov 1 2022 11:30AM,249788,12,"SD Head USA, LLC",1,,
7,Nov 1 2022 11:21AM,249786,10,Methapharm-G,1,4,
8,Nov 1 2022 11:21AM,249786,10,"Methapharm, Inc.",1,4,
9,Nov 1 2022 11:13AM,249783,10,"Citieffe, Inc.",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 1 2022 12:47PM,249799,12,Hush Hush,7,,
1,Nov 1 2022 12:34PM,249798,12,Hush Hush,1,,
2,Nov 1 2022 12:20PM,249797,19,"AdvaGen Pharma, Ltd",9,2,
3,Nov 1 2022 12:05PM,249795,21,"NBTY Global, Inc.",1,,
4,Nov 1 2022 12:05PM,249794,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 1 2022 12:47PM,249799,12,Hush Hush,7.0,NaN,NaN
1,Nov 1 2022 12:34PM,249798,12,Hush Hush,1.0,NaN,NaN
2,Nov 1 2022 12:20PM,249797,19,"AdvaGen Pharma, Ltd",9.0,2.0,NaN
3,Nov 1 2022 12:05PM,249795,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Nov 1 2022 12:05PM,249794,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 1 2022 12:47PM,249799,12,Hush Hush,7.0,0.0,0.0
1,Nov 1 2022 12:34PM,249798,12,Hush Hush,1.0,0.0,0.0
2,Nov 1 2022 12:20PM,249797,19,"AdvaGen Pharma, Ltd",9.0,2.0,0.0
3,Nov 1 2022 12:05PM,249795,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Nov 1 2022 12:05PM,249794,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,999115,7.0,8.0,0.0
12,999159,10.0,0.0,0.0
15,499435,11.0,24.0,89.0
16,499473,1.0,1.0,2.0
19,249797,9.0,2.0,0.0
21,999135,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,999115,7.0,8.0,0.0
1,12,999159,10.0,0.0,0.0
2,15,499435,11.0,24.0,89.0
3,16,499473,1.0,1.0,2.0
4,19,249797,9.0,2.0,0.0
5,21,999135,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,7.0,8.0,0.0
1,12,10.0,0.0,0.0
2,15,11.0,24.0,89.0
3,16,1.0,1.0,2.0
4,19,9.0,2.0,0.0
5,21,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,7.0
1,12,Released,10.0
2,15,Released,11.0
3,16,Released,1.0
4,19,Released,9.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21
Status,,,,,,
Completed,0.0,0.0,89.0,2.0,0.0,0.0
Executing,8.0,0.0,24.0,1.0,2.0,1.0
Released,7.0,10.0,11.0,1.0,9.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21
0,Completed,0.0,0.0,89.0,2.0,0.0,0.0
1,Executing,8.0,0.0,24.0,1.0,2.0,1.0
2,Released,7.0,10.0,11.0,1.0,9.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21
0,Completed,0.0,0.0,89.0,2.0,0.0,0.0
1,Executing,8.0,0.0,24.0,1.0,2.0,1.0
2,Released,7.0,10.0,11.0,1.0,9.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()